In [76]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd
import csv

from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
import asyncio

CODE TEST PROXY

In [77]:
# define custom options for the Selenium driver
options = Options()

# free proxy server URL
# proxy_server_url = "157.245.97.60"
proxy_server_url = '154.6.98.26:3128'
options.add_argument(f'--proxy-server={proxy_server_url}')

# create the ChromeDriver instance with custom options
driver = webdriver.Chrome(options=options)

# driver.get('http://httpbin.org/ip')
# driver.get('https://www.malt.fr/profile/luccharlopeau')

INITIATE PROXY POOL

In [78]:
proxy_pool = [
    '154.6.98.26:3128'
]

# define queue to store the scraped data
from asyncio import Queue

async def append_to_queue(item):
    await queue.put(item)

async def consume_queue():
    while True:
        item = await queue.get()
        print(item)

In [79]:
async def scrap_user(link, proxy):
    """
    Scrap the profile of a user on Malt.fr
    
    Parameters
    ----------
    link : list
        link to scrap.
    proxy : string
        proxy to use.
        
    Returns
    -------
    all_data : dataFrame
        DataFrame containing all the data scrapped.
    """

    # define custom options for the Selenium driver
    options = Options()
    
    options.add_argument(f'--proxy-server={proxy}')

    # create the ChromeDriver instance with custom options
    driver = webdriver.Chrome(options=options)
    
    # chrome_options = webdriver.ChromeOptions()
    # chrome_options.add_argument('--proxy-server=%s' % proxy)
    # driver = uc.Chrome(options=chrome_options, use_subprocess=True)

    wait = WebDriverWait(driver, 6)
    driver.get(link)
    page_source = driver.page_source
    soup = BeautifulSoup(page_source, 'html.parser')
    
    data = {}

    price_element = soup.find('div', {'data-testid': 'profile-price'})
    if price_element:
        price = price_element.find('span', class_='block-list__price').text
        data['price'] = price.strip()
    
    # Récupérer l'expérience
    experience_element = soup.find('span', string='Expérience')
    if experience_element:
        experience = experience_element.find_next('span', class_='profile-indicators-content').text
        data['experience'] = experience.strip()
    
    # Récupérer le taux de réponse
    response_rate_element = soup.find('span', string='Taux de réponse')
    if response_rate_element:
        response_rate = response_rate_element.find_next('span', class_='profile-indicators-content').text
        data['response_rate'] = response_rate.strip()
    
    # Récupérer le temps de réponse
    response_time_element = soup.find('span', string='Temps de réponse')
    if response_time_element:
        response_time = response_time_element.find_next('span', class_='profile-indicators-content').text
        data['response_time'] = response_time.strip()
        
    # Récupérer le nom 
    name_element = soup.find('div', {'data-testid': 'profile-fullname'})
    if name_element:
        name = name_element.text
        data['name'] = name.strip()
        
    # Récupérer le métier
    headline_element = soup.find('div', {'data-testid': 'profile-headline'})
    if headline_element:
        headline = headline_element.text
        data['headline'] = headline.strip()
        
    # Récupérer le nombre de missions
    missions_element = soup.find('div', {'data-testid': 'profile-counter-missions'})
    if missions_element:
        missions = missions_element.find('strong').text
        data['missions'] = missions.strip()
        
    # Récupérer toutes les catégories
    categories_elements = soup.find_all('li', {'class': 'categories__list-item'})
    categories = [category.find('a').text for category in categories_elements]
    data['categories'] = categories
    
    
    # Récupérer les compétences
    competences_element = soup.find_all('div', {'class': 'profile-expertises__content-list-item__label'})
    competences = [competence.find('a', class_='joy-link joy-link_teal').text.strip() for competence in competences_element]

    data['competences'] = competences
    
    # Récupérer le statut "Supermalter"
    supermalter_element = soup.find('span', class_='joy-badge-level__tag blue')
    if supermalter_element:
        supermalter = supermalter_element.get_text(strip=True)
        data['supermalter'] = supermalter
        
    # Récupérer la localisation
    location_element = soup.find('dl', {'class': 'profile__location-and-workplace-preferences__item'})
    if location_element:
        location_label = location_element.find('dt', {'data-testid': 'profile-location-address-label'})
        location_value = location_element.find('dd', {'data-testid': 'profile-location-preference-address'})

        if location_label and location_value:
            location = {location_label.text: location_value.text}
            data['location'] = location
            
    # Récupérer la préférence de télétravail
    teletravail_element = soup.find('dl', {'class': 'profile-page-mission-preferences__item'})
    if teletravail_element:
        teletravail_label = teletravail_element.find('dt')
        teletravail_value = teletravail_element.find('dd')

        if teletravail_label and teletravail_value:
            teletravail_preference = {teletravail_label.text: teletravail_value.text}
            data['teletravail_preference'] = teletravail_preference
            
    # Récupérer le nombre de recommandations
    recommendations_element = soup.find('span', {'data-testid': 'profile-counter-recommendations'})
    if recommendations_element:
        recommendations_count = int(recommendations_element.text.split()[0])
        data['recommendations'] = recommendations_count   
        

    # Récupérer le message de présentation
    presentation_element = soup.find('div', {'class': 'profile-description__content'})
    if presentation_element:
        presentation_message = presentation_element.get_text(strip=True)
        data['presentation'] = presentation_message
          
    driver.quit() # close the browser
    # time.sleep(5) # wait for 5 seconds to avoid getting banned
    
    return data # return the data scrapped

In [80]:
async def scrap_all_users_proxy(links): # links is a list of links to scrap.
    """
    Scrap the profile of every link in the list of links.
    
    Parameters
    ----------
    links : list
        List of links to scrap.
        
    Returns
    -------
    all_data : dataFrame
        DataFrame containing all the data scrapped.
    """
    all_data = [] # list to store all the data scrapped
    
    queue = Queue()

    for link in links: # loop over all the links
        scrapped_user = await scrap_user(link, proxy_pool[0]) # scrap the user
        print(scrapped_user)
        
        await queue.put(scrapped_user)  # Asynchronously append to the queue


        while not queue.empty():  # Consume the queue
            scrapped_user = await queue.get()
            all_data.append(scrapped_user)  # Append to the data list
    
    return all_data # return the list of data scrapped

RUN THE SCRIPT WITH THE LINKS

In [81]:
profile_links = pd.read_csv('../data/links.csv')
profile_links['profil'] = profile_links['profil'].apply(lambda x: x.replace('https://www.malt.fr/profile/', ''))
profile_links = profile_links['profil'].tolist()
profile_links = [f'https://www.malt.fr/profile/{link}' for link in profile_links] # create the links to scrap

async def main():
    task = asyncio.create_task (scrap_all_users_proxy(profile_links[:3]))
    await task
    return task.result()
data = await main()

NameError: name 'time' is not defined

In [ ]:
data

[]

SAVE THE DATA TO CSV

In [ ]:
def save_to_csv(data, csv_filename):
    # Assurez-vous que la liste de données n'est pas vide
    if not data:
        print("Aucune donnée à enregistrer.")
        return

    # Créez une liste de noms de colonnes pour le CSV en incluant les nouveaux éléments
    fieldnames = ['name', 'headline', 'price', 'experience', 'response_rate', 'response_time', 'missions', 'categories', 'competences', 'supermalter', 'location','presentation', 'recommendations', 'teletravail_preference']

    # Ouvrir le fichier CSV en mode écriture
    with open(csv_filename, 'w', newline='', encoding='utf-8') as csv_file:
        writer = csv.DictWriter(csv_file, fieldnames=fieldnames)

        # Écrire les en-têtes
        writer.writeheader()

        # Écrire les données
        for entry in data:
            writer.writerow(entry)

    print(f"Données enregistrées dans {csv_filename}")

In [ ]:
csv_filename = 'malt_data.csv'
save_to_csv(data, csv_filename)

Aucune donnée à enregistrer.
